# Import libraries 

In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from scipy.optimize import minimize, fmin_slsqp
from datetime import date, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pylab
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, KFold
import lightgbm as lgb 

# Import data 

In [2]:
%%time
# train and sub format already merged with weather and holidays (plus the day before holiday)
train = pd.read_csv('train.csv')
hol = pd.read_csv('hol.csv')
hol['Timestamp'] = pd.to_datetime(hol['Timestamp'])

CPU times: user 555 ms, sys: 71.2 ms, total: 626 ms
Wall time: 980 ms


## gold and usd preprocess

In [3]:
gold = pd.read_csv('gold_usd/gold.csv')
usd = pd.read_csv('gold_usd/usd.csv')

FileNotFoundError: File b'gold_usd/gold.csv' does not exist

In [ ]:
gold['DATE'] = pd.to_datetime(gold['DATE'])
usd['DATE'] = pd.to_datetime(usd['DATE'])

In [ ]:
print(gold.shape)
gold = gold[(gold.TICKER==1) & (gold.DATE>=pd.to_datetime('2015-01-01')) & (gold.DATE<=pd.to_datetime('2017-09-17'))]
gold = gold[['DATE','CLOSE']].reset_index(drop=True)
print(gold.shape)

In [ ]:
usd.head()

In [ ]:
print(usd.shape)
usd = usd[(usd.DATE>=pd.to_datetime('2015-01-01')) & (usd.DATE<=pd.to_datetime('2017-09-17'))]
usd = usd[(usd.TICKER=='USD') | (usd.TICKER=='EUR')]
usd = usd[['DATE','CLOSE','TICKER']].reset_index(drop=True)
print(gold.shape)

In [ ]:
usd = usd.pivot(index='DATE', columns='TICKER', values='CLOSE')
usd = usd.reset_index()

In [ ]:
min_date = pd.to_datetime('2015-01-01')
max_date = pd.to_datetime('2017-09-17')
dates = pd.date_range(start=min_date, end=max_date, periods=None, freq='D')
h = pd.DataFrame({'DATE': dates})

In [ ]:
h = pd.merge(h, usd, how='left', on='DATE')
h = pd.merge(h, gold, how='left', on='DATE')

In [ ]:
h.columns = ['Timestamp','EUR','USD','CLOSE']

In [ ]:
for i in ['EUR', 'USD', 'CLOSE']:
    h[i] = h[i].interpolate()

In [ ]:
new_cols = []
for i in [1,7,14,30]:
    for j in ['EUR', 'USD', 'CLOSE']:
        new_col = j + '_' + str(i)
        new_cols.append(new_col)
        h[new_col] = h[j].diff(i)

In [ ]:
new_cols

In [ ]:
h = h.fillna(0)
h.to_csv('golds.csv',index=False)

## validate 

In [ ]:
def MeanEncodingTransforming(X, y, X_test, how_to_fill):
    
    # mean encoding for lgb
    
    X_train = pd.concat([X, y], axis=1)
    mean_values = X_train.groupby(X_train.columns[0]).agg(how_to_fill).to_dict()['Value']
    X_train = X_train.drop(y.columns[0], axis=1)
    X_train = X_train.replace(mean_values)
    X_test = X_test.replace(mean_values)
    
    return X_train, X_test

def feature_preprocessing(train, test, cat_cols, cat_type='ohe'):
    
    # ohe or mean encoding preprocessing for the lgb 
    
    X_train, X_test = train[cat_cols].copy(), test[cat_cols].copy()

    if (cat_type=='mean_enc'):
        for j in ['mean', 'max', 'min', 'median']:
            for i in cat_cols:
                X_train[i], X_test[i] = MeanEncodingTransforming(X_train[i], train[['Value']], X_test[i], j)
                X_train[i].columns = [i+'_'+j]
                X_test[i].columns = [i+'_'+j]
                
    if (cat_type=='ohe'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        X_train = ohe.fit_transform(train[cat_cols])
        X_test = ohe.transform(test[cat_cols])
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        
    X_train = pd.concat([X_train, train[['days_to_holiday', 'days_after_holiday']]], axis=1).fillna(-999)
    X_test = pd.concat([X_test, test[['days_to_holiday', 'days_after_holiday']]], axis=1).fillna(-999)
    
    return X_train, X_test 

In [ ]:
def train_groupby(train, test, window, how):
    
    # simple groupby prediction 
    
    # time_delta = list((test['Timestamp'].iloc[-1:]  - train['Timestamp'].iloc[1] ).dt.days)[0]

    mean_values = train[['Value', 'week_day']][-window:].groupby(['week_day']).agg(how).reset_index()
    mean_values.columns = ['week_day', 'pred']
    test = pd.merge(test, mean_values, how='left', on = ['week_day'])  
    
    return test['pred'].fillna(np.mean(train['Value']))

def train_mean(train, window):
    
    # return mean value from train for the window 
    
    mean_value = np.mean(train['Value'].reset_index(drop=True)[-window:])
   
    return mean_value


In [ ]:
def validate_lgb(X_train, y_train, X_valid, y_valid):
    
    
    d1 = lgb.Dataset(X_train, y_train, weight=np.linspace(0.5, 1, X_train.shape[0]))
    d2 = lgb.Dataset(X_valid, y_valid)
    
    params = {
        'objective':'regression',    
        'metric': 'l1', 
        'learning_rate': 0.160042,
        'random_state':42,
        'min_data':1,
        'min_data_in_bin':1
    }
    
    gbm = lgb.train(params, d1, verbose_eval=None, valid_sets=d2, 
                    num_boost_round=50000, early_stopping_rounds=100)
    
    y_hat = gbm.predict(X_valid)
    opt_boost_rounds = gbm.best_iteration
    
    return y_hat, opt_boost_rounds 



def train_lgb(X_train, y_train, X_test, opt_boost_rounds):
    
    d1 = lgb.Dataset(X_train, y_train, weight=np.linspace(0.5, 1, X_train.shape[0]))
    
    params = {
        'objective':'regression',    
        'metric': 'l1', 
        'learning_rate': 0.160042,
        'random_state':42,
        'min_data':1,
        'min_data_in_bin':1
    }
    
    gbm = lgb.train(params, d1, verbose_eval=None, num_boost_round=opt_boost_rounds)
    
    y_hat = gbm.predict(X_test)

    return y_hat

In [ ]:
def calc_score(pred, fact, index_mult):
    return np.sum(abs(pred-fact)) / np.sum(fact) * 10000

def train_rf(X_train, y_train, X_valid):

    rf = RandomForestRegressor(max_features='sqrt', n_estimators=142, n_jobs=-1, random_state=4224)
    rf.fit(X_train, y_train, sample_weight=np.linspace(0.5, 1, X_train.shape[0]) )
    y_hat = rf.predict(X_valid)
    
    return y_hat

In [ ]:
def combine_y_hats(y_hats):
    X_stack = pd.DataFrame({})
    for i in range(0, len(y_hats)):
        X_stack['stack'+str(i)] = y_hats[i]
    return X_stack

In [ ]:
def train_stack(X_stack, y, model):
    model.fit(X_stack, y)
    return model

In [ ]:
def make_harmonic_features(x, col, period=24):
    x['sin_'+col] = np.sin(x[col] * 2 * np.pi / period)
    x['cos_'+col] = np.cos(x[col] * 2 * np.pi / period)
    x = x.drop(col, axis=1)
    return x

## predict

In [ ]:
train['minutes_in_day'] = train['hour']*60 + train['minute']

In [ ]:
train['holidays'] = 0

In [ ]:
train.head()

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False, random_state=None)

def validate_stack(X_stack, y, model):
    y_cros_val_pred = cross_val_predict(model, X_stack, y=y, cv=5, n_jobs=-1)
    return y_cros_val_pred

def cv_lr(X_stack, y, model):
    
    # cros val lasso on X_stack 
    
    y_cros_val_pred = pd.DataFrame({})
    coefs = []
    
    for train_index, test_index in kf.split(X_stack):
        X_train, X_test = X_stack.iloc[train_index], X_stack.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model_temp = model 
        model_temp.fit(X_train,y_train)
        y_hat = model_temp.predict(X_test)
        coefs.append(model_temp.coef_)
        
        temp_df = pd.DataFrame({'id': test_index, 'Value': y_hat})
        y_cros_val_pred = pd.concat([y_cros_val_pred, temp_df], axis=0)
      
    y_cros_val_pred = y_cros_val_pred.sort_values(by='id')
    
    #divide by sum - due to overfit of model, with dividing - coefs ~ weights of models
    return list(y_cros_val_pred['Value']) / (np.sum(np.mean(coefs,axis=0)) + 1)

In [ ]:
# train.columns = ['Timestamp','ForecastId','Value']
def time_preprocess(X):
    X['Timestamp'] = pd.to_datetime(X['Timestamp'])
    X['year'] = X['Timestamp'].dt.year
    X['month'] = X['Timestamp'].dt.month 
    X['week'] = X['Timestamp'].dt.week
    X['day'] = X['Timestamp'].dt.day
    X['week_day'] = X['Timestamp'].dt.weekday
    X['hour'] = X['Timestamp'].dt.hour
    X['minute'] = X['Timestamp'].dt.minute
    X['minute'] = X['minute'] // 15 * 15
    
    return X

val = train[['ForecastId','Value']].groupby('ForecastId').diff()
val['Value'] = val['Value'].fillna(0)
val['Value'] = (val['Value']==0) * 1
val.columns = ['diff']
train = pd.concat([train, val], axis=1)

a = train[['ForecastId','diff']].groupby('ForecastId').apply(pd.rolling_sum, 7, min_periods=1)
a.columns = ['ForecastId', 'to_drop']
a = a['to_drop']
train = pd.concat([train, a], axis=1)

print(train.shape)
train = train[train['to_drop']<=3].reset_index(drop=True)
train = train[['Timestamp','ForecastId','Value']]
print(train.shape)



train = time_preprocess(train)
train = train[train.year>=2016].reset_index(drop=True)

#max_values = train[['Value', 'ForecastId']].groupby('ForecastId').agg('max').reset_index()
#max_values.columns = ['ForecastId', 'max_value']
#train = pd.merge(train, max_values, on = 'ForecastId', how='left')
#train['Value'] = train['Value'] / train['max_value']
#train = train.drop('max_value',axis=1)

train['year_month_weekday'] = train['year'].astype('str') + '_' + train['month'].astype('str') + '_' + train['week_day'].astype('str') 
features = train[['ForecastId', 'year_month_weekday', 'Value']].groupby(['ForecastId', 'year_month_weekday']).agg('mean').reset_index()
features = features.pivot(index='ForecastId', columns='year_month_weekday', values='Value')

features = features.fillna(0)
train = pd.merge(train, hol, on='Timestamp', how='left')

In [ ]:
%%time

losses = []
iids = []

sub = pd.DataFrame({})
cat_cols = ['week_day','month','year', 'week']

pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']

for i in pd.unique(train['ForecastId'])[0:50]:
 
    rf_stack = RandomForestRegressor(n_estimators=1000, random_state=42, n_jobs=-1)
    lr_stack = Lasso(alpha=1, fit_intercept=False, max_iter=3000, tol=0.0001, positive=True, random_state=424142)

    # prepare train and test Dfs 
    days_in_train = 99999
    X_train = train[train['ForecastId']==i].reset_index(drop=True)[-days_in_train:].reset_index(drop=True)
    X_train['Value'] = abs(X_train['Value'])
    X_train['Value_log'] = np.log1p(X_train['Value'])
    
    # drop outliers in train data set 
    up_border = X_train['Value'].quantile(0.985)
    low_border = X_train['Value'].quantile(0.015)
    X_train = X_train[(X_train['Value']<=up_border) & (X_train['Value']>=low_border)].reset_index(drop=True)
    
    # prepare 'train_v' and 'valid_v' data frames - for validation 
    X_test = pd.DataFrame({'Timestamp':pred_dates, 'ForecastId':i})
    X_test = time_preprocess(X_test)
    X_test = pd.merge(X_test, hol, on='Timestamp', how='left')
    
    obs_in_test = 35
    X_train_v = X_train[:-obs_in_test].reset_index(drop=True)
    X_valid_v = X_train[-obs_in_test:].reset_index(drop=True)
    
    # prepare features 
        # for train
    X_train_ohe, X_test_ohe = feature_preprocessing(X_train, X_test, cat_cols, cat_type='ohe')
    X_train_meanenc, X_test_meanenc = feature_preprocessing(X_train, X_test, cat_cols, cat_type='mean_enc')
    
        # for validation 
    X_train_v_ohe, X_valid_v_ohe = feature_preprocessing(X_train_v, X_valid_v, cat_cols, cat_type='ohe')
    X_train_v_meanenc, X_valid_v_meanenc = feature_preprocessing(X_train_v, X_valid_v, cat_cols, cat_type='mean_enc')
    
        # group by mean
    y_hat_grby_mean = train_groupby(X_train_v, X_valid_v, window=999999, how='mean')
    y_hat_grby3_mean = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='mean')
    
        # group by median
    y_hat_grby_median = train_groupby(X_train_v, X_valid_v, window=999999, how='median')
    y_hat_grby3_median = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='median')
    
        # RandomForest 
    y_rf = train_rf(X_train_v_ohe, X_train_v['Value'], X_valid_v_ohe)
    y_rf_mean = train_rf(X_train_v_meanenc, X_train_v['Value'], X_valid_v_meanenc)
    y_rf_log = np.exp(train_rf(X_train_v_ohe, X_train_v['Value_log'], X_valid_v_ohe)) - 1
    y_rf_mean_log = np.exp(train_rf(X_train_v_meanenc, X_train_v['Value_log'], X_valid_v_meanenc)) - 1
    
        # LightGBM
    #y_lgb, lgb_opt = validate_lgb(X_train_v_ohe, X_train_v['Value'], X_valid_v_ohe, X_valid_v['Value'])
    #y_lgb_mean, lgb_mean_opt = validate_lgb(X_train_v_meanenc, X_train_v['Value'], X_valid_v_meanenc, X_valid_v['Value'])
    #y_lgb_log, lgb_opt_log = validate_lgb(X_train_v_ohe, X_train_v['Value_log'], X_valid_v_ohe, X_valid_v['Value_log'])
    #y_lgb_log = np.exp(y_lgb_log) -1
    #y_lgb_mean_log, lgb_mean_opt_log = validate_lgb(X_train_v_meanenc, X_train_v['Value_log'], X_valid_v_meanenc, X_valid_v['Value_log'])
    #y_lgb_mean_log = np.exp(y_lgb_mean_log) -1
    
    # stack predictions and make predictions 
    X_valid_stack = combine_y_hats([y_hat_grby_mean, y_hat_grby3_mean, 
                                    y_hat_grby_median, y_hat_grby3_median, 
                                    y_rf, y_rf_mean, 
                                    y_rf_log, y_rf_mean_log, 
                                    #y_lgb, y_lgb_mean, 
                                   #y_lgb_log, y_lgb_mean_log
                                   ])
    #y_rf_hat = validate_stack(X_valid_stack, X_valid_v['Value'], rf_stack)
    #y_lr_hat = cv_lr(X_valid_stack, X_valid_v['Value'], lr_stack)
 
    # calculate scores and pick top model 
    iid = X_valid_v.reset_index()['index'] 
    T = np.max(iid)
    index_mult = (3*T -2*iid +1) / 2 / T**2
    
    score_grby_mean = calc_score(y_hat_grby_mean, X_valid_v['Value'], index_mult)
    score_grby3_mean = calc_score(y_hat_grby3_mean, X_valid_v['Value'], index_mult)
    score_grby_median = calc_score(y_hat_grby_median, X_valid_v['Value'], index_mult)
    score_grby3_median = calc_score(y_hat_grby3_median, X_valid_v['Value'], index_mult)
    
    score_rf = calc_score(y_rf, X_valid_v['Value'], index_mult)
    score_rf_mean = calc_score(y_rf_mean, X_valid_v['Value'], index_mult)
    score_rf_log = calc_score(y_rf_log, X_valid_v['Value'], index_mult)
    score_rf_mean_log = calc_score(y_rf_mean_log, X_valid_v['Value'], index_mult)
    
    #score_lgb = calc_score(y_lgb, X_valid_v['Value'], index_mult)
    #score_lgb_mean = calc_score(y_lgb_mean, X_valid_v['Value'], index_mult)
    #score_lgb_log = calc_score(y_lgb_log, X_valid_v['Value'], index_mult)
    #score_lgb_mean_log = calc_score(y_lgb_mean_log, X_valid_v['Value'], index_mult)

    #score_lr_stack = calc_score(y_lr_hat, X_valid_v['Value'], index_mult)
    #score_rf_stack = calc_score(y_rf_hat, X_valid_v['Value'], index_mult)
    
    
    y_hats     = [y_hat_grby_mean, y_hat_grby3_mean, 
                  y_hat_grby_median, y_hat_grby3_median, 
                  y_rf, 
                  y_rf_mean, 
                  y_rf_log, 
                  y_rf_mean_log, 
                  #y_lgb, y_lgb_mean, 
                  #y_lgb_log, y_lgb_mean_log, 
                  #y_lr_hat, y_rf_hat
                 ]
    
    all_scores = [score_grby_mean, score_grby3_mean, 
                  score_grby_median, score_grby3_median, 
                  score_rf, 
                  score_rf_mean, 
                  score_rf_log, 
                  score_rf_mean_log, 
                  #score_lgb, score_lgb_mean, 
                  #score_lgb_log, score_lgb_mean_log, 
                  #score_lr_stack
                 ]
    
    best_score = np.min(all_scores)
    
    models_names = ['mean_all', 'mean_3', 
                    'median_all', 'median_3', 
                    'rf', 'rf_mean', 
                    'rf_log', 'rf_mean_log', 
                    #'lgb', 'lgb_mean', 
                    #'lgb_log', 'lgb_mean_log', 
                    #'lr_stack', 'rf_stack'
                   ]
    # plot figures and seve to folder 
    fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
    ax.plot(y_hats[np.argmin(all_scores)])
    ax.plot(X_valid_v['Value'].reset_index(drop=True))
    fig.savefig('C:/Users/denis/Machine_Learning_Competitions/idao/lr_stack/'+str(i)+'.jpg')   # save the figure to file
    # plt.show()
    plt.close(fig)    # close the figure
    
    #calc R2 and save later 
    #r2 = r2_score(y_hats[np.argmin(all_scores)], X_valid_v['Value'] ) 
    #losses.append( r2 )
    
    losses.append(best_score)
    
    temp_df = pd.DataFrame({'Timestamp':pred_dates, 'ForecastId':i})
    temp_df = time_preprocess(temp_df)
    
    if (score_grby_mean==best_score):
        y_hat = train_groupby(X_train, X_test, window=999999, how='mean')
    
    if (score_grby3_mean==best_score):
        y_hat = train_groupby(X_train, X_test, window=obs_in_test*3, how='mean')
        
    if (score_grby_median==best_score):
        y_hat = train_groupby(X_train, X_test, window=999999, how='median')
    
    if (score_grby3_median==best_score):
        y_hat = train_groupby(X_train, X_test, window=obs_in_test*3, how='median')
    y_hat = 0 
    X_test['Value'] = y_hat
    sub = pd.concat([sub,X_test],axis=0)
    
  
    print(i, 'Best model is', models_names[np.argmin(all_scores)])
    print('loss:', np.min(all_scores) )
    print('val score:', np.mean(losses),'+',np.std(losses) )
    print(dict(zip(models_names, all_scores)))
    print('-------------------------------')


In [ ]:
60 Best model is mean_all
loss: 4305.94860943
val score: 4158.48998272 + 2005.10425588
{'rf_log': 4946.362227144722, 'median_3': 6169.4187582562745, 'rf_mean_log': 5304.017089875222, 'mean_3': 5520.169214317167, 'median_all': 4584.742404227213, 'mean_all': 4305.948609431632, 'rf_mean': 4971.130659230724, 'rf': 4742.6985121311545}

## sub 

In [ ]:
%%time

import pandas as pd 
import numpy as np 
from datetime import date, timedelta
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Lasso, ElasticNet, LinearRegression

train = pd.read_csv('train.csv')
hol = pd.read_csv('hol.csv')
h = pd.read_csv('golds.csv')
hol['Timestamp'] = pd.to_datetime(hol['Timestamp'])
h['Timestamp'] = pd.to_datetime(h['Timestamp'])

def time_preprocess(X):
    X['Timestamp'] = pd.to_datetime(X['Timestamp'])
    X['year'] = X['Timestamp'].dt.year
    X['month'] = X['Timestamp'].dt.month 
    X['day'] = X['Timestamp'].dt.day
    X['week_day'] = X['Timestamp'].dt.weekday
    X['hour'] = X['Timestamp'].dt.hour
    X['minute'] = X['Timestamp'].dt.minute
    X['minute'] = X['minute'] // 15 * 15
    
    return X
train.columns = ['Timestamp','ForecastId','Value']
train = time_preprocess(train)
train = train[train.year>=2016].reset_index(drop=True)

def feature_preprocessing(train, test, cat_cols, cat_type='ohe'):
    
    # ohe or mean encoding preprocessing for the lgb 
    
    X_train, X_test = train[cat_cols].copy(), test[cat_cols].copy()

    if (cat_type=='mean_enc'):
        for j in ['mean', 'max', 'min', 'median']:
            for i in cat_cols:
                X_train[i], X_test[i] = MeanEncodingTransforming(X_train[i], train[['Value']], X_test[i], j)
                X_train[i].columns = [i+'_'+j]
                X_test[i].columns = [i+'_'+j]
                
    if (cat_type=='ohe'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        X_train = ohe.fit_transform(train[cat_cols])
        X_test = ohe.transform(test[cat_cols])
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        
    if (cat_type=='harmonic'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        temp_train = ohe.fit_transform(train[['year']])
        temp_test = ohe.transform(test[['year']])
        X_train = pd.concat([X_train, pd.DataFrame(temp_train)],axis=1)
        X_test = pd.concat([X_test, pd.DataFrame(temp_test)],axis=1)
        for i in range(len(cat_cols)):
            X_train = make_harmonic_features(X_train, cat_cols[i], period=harmonic_features_max[i])
            X_test = make_harmonic_features(X_test, cat_cols[i], period=harmonic_features_max[i])
        
    #X_train = pd.concat([X_train, train[new_cols]], axis=1).fillna(-999)
    #X_test = pd.concat([X_test, test[new_cols]], axis=1).fillna(-999)
    
    return X_train, X_test 

def train_groupby(train, test, window, how):
    mean_values = train[['Value', 'week_day']][-window:].groupby(['week_day']).agg(how).reset_index()
    mean_values.columns = ['week_day', 'pred']
    test = pd.merge(test, mean_values, how='left', on = ['week_day'])  
    
    return test['pred'].fillna(np.mean(train['Value']))

def train_mean(train, window):
    mean_value = np.mean(train['Value'].reset_index(drop=True)[-window:])
    return mean_value

def calc_score(pred, fact, index_mult):
    return np.sum(abs(pred-fact)) / np.sum(fact) * 10000

def train_rf(X_train, y_train, X_valid):

    rf = RandomForestRegressor(max_features='sqrt', n_estimators=180, n_jobs=-1, random_state=4224)
    rf.fit(X_train, y_train, sample_weight=np.linspace(0.5, 1, X_train.shape[0]) )
    y_hat = rf.predict(X_valid)
    
    return y_hat


# train.columns = ['Timestamp','ForecastId','Value']
def time_preprocess(X):
    X['Timestamp'] = pd.to_datetime(X['Timestamp'])
    X['year'] = X['Timestamp'].dt.year
    X['month'] = X['Timestamp'].dt.month 
    X['week'] = X['Timestamp'].dt.week
    X['day'] = X['Timestamp'].dt.day
    X['week_day'] = X['Timestamp'].dt.weekday
    
    return X

val = train[['ForecastId','Value']].groupby('ForecastId').diff()
val['Value'] = val['Value'].fillna(0)
val['Value'] = (val['Value']==0) * 1
val.columns = ['diff']
train = pd.concat([train, val], axis=1)

a = train[['ForecastId','diff']].groupby('ForecastId').apply(pd.rolling_sum, 7, min_periods=1)
a.columns = ['ForecastId', 'to_drop']
a = a['to_drop']
train = pd.concat([train, a], axis=1)

print(train.shape)
train = train[train['to_drop']<=3].reset_index(drop=True)
train = train[['Timestamp','ForecastId','Value']]
train = time_preprocess(train)
print(train.shape)

train = pd.merge(train, hol, on='Timestamp', how='left')
train = pd.merge(train, h, on='Timestamp', how='left')
losses = []
iids = []

sub = pd.DataFrame({})
cat_cols = ['year','week_day','month','week']
harmonic_features = ['week_day','month','week']
harmonic_features_max = [6, 12, 53]

pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']

def combine_y_hats(y_hats):
    X_stack = pd.DataFrame({})
    for i in range(0, len(y_hats)):
        X_stack['stack'+str(i)] = y_hats[i]
    return X_stack

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False, random_state=None)

def validate_stack(X_stack, y, model):
    y_cros_val_pred = cross_val_predict(model, X_stack, y=y, cv=5, n_jobs=-1)
    return y_cros_val_pred

def cv_lr(X_stack, y, model):
    
    # cros val lasso on X_stack 
    
    y_cros_val_pred = pd.DataFrame({})
    coefs = []
    
    for train_index, test_index in kf.split(X_stack):
        X_train, X_test = X_stack.iloc[train_index], X_stack.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model_temp = model 
        model_temp.fit(X_train,y_train)
        y_hat = model_temp.predict(X_test)
        coefs.append(model_temp.coef_)
        
        temp_df = pd.DataFrame({'id': test_index, 'Value': y_hat})
        y_cros_val_pred = pd.concat([y_cros_val_pred, temp_df], axis=0)
      
    y_cros_val_pred = y_cros_val_pred.sort_values(by='id')
    
    #divide by sum - due to overfit of model, with dividing - coefs ~ weights of models
    return list(y_cros_val_pred['Value']) / np.sum(np.mean(coefs,axis=0)) 

def make_harmonic_features(x, col, period=24):
    x['sin_'+col] = np.sin(x[col] * 2 * np.pi / period)
    x['cos_'+col] = np.cos(x[col] * 2 * np.pi / period)
    x = x.drop(col, axis=1)
    return x

index_mult = 0

for i in pd.unique(train['ForecastId'])[0:50]:
    lr_stack = Lasso(alpha=1, fit_intercept=False, max_iter=3000, tol=0.0001, positive=True, random_state=424142)
    
    # prepare train and test Dfs 
    days_in_train = 9999
    X_train = train[train['ForecastId']==i].reset_index(drop=True)[-days_in_train:].reset_index(drop=True)
    up_border = X_train['Value'].quantile(0.985)
    low_border = X_train['Value'].quantile(0.015)
    X_train = X_train[(X_train['Value']<=up_border) & (X_train['Value']>=low_border)].reset_index(drop=True)
    
    X_train['Value'] = abs(X_train['Value']) 
    X_train['Value_log'] = np.log1p(X_train['Value'])
     
    # prepare 'train_v' and 'valid_v' data frames - for validation 
    X_test = pd.DataFrame({'Timestamp':pred_dates, 'ForecastId':i})
    X_test = time_preprocess(X_test)
    #X_test = pd.merge(X_test, hol, on='Timestamp', how='left')
    #X_test = pd.merge(X_test, h, on='Timestamp', how='left')
    
    obs_in_test = 35
    X_train_v = X_train[:-obs_in_test].reset_index(drop=True)
    X_valid_v = X_train[-obs_in_test:].reset_index(drop=True)
    
    # prepare features 
        # for train
    X_train_ohe, X_test_ohe = feature_preprocessing(X_train, X_test, cat_cols, cat_type='ohe') 
    X_train_harm, X_test_harm = feature_preprocessing(X_train, X_test, harmonic_features, cat_type='harmonic')
  
        # for validation 
    X_train_v_ohe, X_valid_v_ohe = feature_preprocessing(X_train_v, X_valid_v, cat_cols, cat_type='ohe')
    X_train_v_harm, X_valid_v_harm = feature_preprocessing(X_train_v, X_valid_v, harmonic_features, cat_type='harmonic')
    
        # group by mean
    y_hat_grby_mean = train_groupby(X_train_v, X_valid_v, window=999999, how='mean')
    y_hat_grby3_mean = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='mean')
    
        # group by median
    y_hat_grby_median = train_groupby(X_train_v, X_valid_v, window=999999, how='median')
    y_hat_grby3_median = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='median')
    
        # RandomForest 
    y_rf = train_rf(X_train_v_ohe, X_train_v['Value'], X_valid_v_ohe)
    y_rf_harm = train_rf(X_train_v_harm, X_train_v['Value'], X_valid_v_harm)
    y_rf_log = np.exp(train_rf(X_train_v_ohe, X_train_v['Value_log'], X_valid_v_ohe)) - 1
    y_rf_log_harm = np.exp(train_rf(X_train_v_harm, X_train_v['Value_log'], X_valid_v_harm)) - 1
    
    X_valid_stack = combine_y_hats([y_hat_grby_mean, y_hat_grby3_mean, 
                                    y_hat_grby_median, y_hat_grby3_median, 
                                    y_rf, y_rf_harm, 
                                    y_rf_log, y_rf_log_harm, 
                                    
                                   # y_lgb, y_lgb_mean, 
                                   # y_lgb_log, y_lgb_mean_log
                                   ])
    y_lr_hat = cv_lr(X_valid_stack, X_valid_v['Value'], lr_stack)
    
    score_grby_mean = calc_score(y_hat_grby_mean, X_valid_v['Value'], 0)
    score_grby3_mean = calc_score(y_hat_grby3_mean, X_valid_v['Value'], 0)
    score_grby_median = calc_score(y_hat_grby_median, X_valid_v['Value'], 0)
    score_grby3_median = calc_score(y_hat_grby3_median, X_valid_v['Value'], 0)
    
    score_rf = calc_score(y_rf, X_valid_v['Value'], index_mult)
    score_rf_harm = calc_score(y_rf_harm, X_valid_v['Value'], index_mult)
    score_rf_log = calc_score(y_rf_log, X_valid_v['Value'], index_mult)
    score_rf_log_harm = calc_score(y_rf_log_harm, X_valid_v['Value'], index_mult)
    
    score_lr_stack = calc_score(y_lr_hat, X_valid_v['Value'], index_mult)

    y_hats     = [y_hat_grby_mean, y_hat_grby3_mean, 
                  y_hat_grby_median, y_hat_grby3_median, 
                  y_rf, y_rf_harm,
                  y_rf_log, y_rf_log_harm,
                  y_lr_hat
                 ]
    
    all_scores = [score_grby_mean, score_grby3_mean, 
                  score_grby_median, score_grby3_median, 
                  score_rf, score_rf_harm,
                  score_rf_log, score_rf_log_harm,
                  score_lr_stack
                 ]
    
    best_score = np.min(all_scores)
    
    models_names = ['mean_all', 'mean_3', 
                    'median_all', 'median_3', 
                    'rf', 'rf_harm',
                    'rf_log', 'rf_log_harm',
                    'lr_stack'
                   ]
    
    losses.append(best_score)
    
    temp_df = pd.DataFrame({'Timestamp':pred_dates, 'ForecastId':i})
    temp_df = time_preprocess(temp_df)
    
    if (score_grby_mean==best_score):
        y_hat = train_groupby(X_train, X_test, window=999999, how='mean')
    
    if (score_grby3_mean==best_score):
        y_hat = train_groupby(X_train, X_test, window=obs_in_test*3, how='mean')
        
    if (score_grby_median==best_score):
        y_hat = train_groupby(X_train, X_test, window=999999, how='median')
    
    if (score_grby3_median==best_score):
        y_hat = train_groupby(X_train, X_test, window=obs_in_test*3, how='median')

    if (score_rf==best_score):
        y_hat = train_rf(X_train_ohe, X_train['Value'], X_test_ohe)
        
    if (score_rf_log==best_score):
        y_hat = np.exp(train_rf(X_train_ohe, X_train['Value_log'], X_test_ohe)) -1    
        
    X_test['Value'] = y_hat
    sub = pd.concat([sub,X_test],axis=0)
    
  
    print(i, 'Best model is', models_names[np.argmin(all_scores)])
    # print('loss:', np.min(all_scores) )
    print('val score:', np.mean(losses),'+',np.std(losses) )
    print(dict(zip(models_names, all_scores)))
    print('-------------------------------')

sub = sub[['Timestamp', 'ForecastId', 'Value']]
sub.columns = ['DATE', 'ATM_ID', 'CLIENT_OUT']
sub.to_csv('submission.csv', index=False)

In [ ]:
# with days to holiday 


# with holidays 
60 Best model is mean_all
val score: 4139.33460426 + 2082.87253026
{'mean_all': 4301.028429876416, 'rf_mean': 4383.672882474736, 'median_all': 4595.50858652576, 
 'rf': 4784.1237340378675, 'mean_3': 5520.169214317167, 'median_3': 6169.4187582562745}


# no holidays 
60 Best model is mean_all
val score: 4086.31176171 + 1982.26470997
{'mean_all': 4301.028429876416, 'rf_mean': 4306.659144071697, 'median_all': 4595.50858652576, 'rf': 4675.956993982094, 'mean_3': 5520.169214317167, 'median_3': 6169.4187582562745}

In [ ]:
419 Best model is mean_3
val score: 3935.81800742 + 1661.80843643

In [ ]:
%%time

import pandas as pd 
import numpy as np 
from datetime import date, timedelta
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Lasso, ElasticNet, LinearRegression
import lightgbm as lgb

train = pd.read_csv('train.csv')
#hol = pd.read_csv('hol.csv')
#h = pd.read_csv('golds.csv')
#hol['Timestamp'] = pd.to_datetime(hol['Timestamp'])
#h['Timestamp'] = pd.to_datetime(h['Timestamp'])

def time_preprocess(X):
    X['Timestamp'] = pd.to_datetime(X['Timestamp'])
    X['year'] = X['Timestamp'].dt.year
    X['month'] = X['Timestamp'].dt.month 
    X['week'] = X['Timestamp'].dt.week
    X['day'] = X['Timestamp'].dt.day
    X['week_day'] = X['Timestamp'].dt.weekday
    return X
train.columns = ['Timestamp','ForecastId','Value']
train = time_preprocess(train)
train = train[train.year>=2016].reset_index(drop=True)

def feature_preprocessing(train, test, cat_cols, cat_type='ohe'):
    
    # ohe or mean encoding preprocessing for the lgb 
    
    X_train, X_test = train[cat_cols].copy(), test[cat_cols].copy()

    if (cat_type=='mean_enc'):
        for j in ['mean', 'max', 'min', 'median']:
            for i in cat_cols:
                X_train[i+'_'+j], X_test[i+'_'+j] = MeanEncodingTransforming(X_train[i], train[['Value']], X_test[i], j)
                
    if (cat_type=='ohe'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        X_train = ohe.fit_transform(train[cat_cols])
        X_test = ohe.transform(test[cat_cols])
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        
    if (cat_type=='harmonic'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        temp_train = ohe.fit_transform(train[['year']])
        temp_test = ohe.transform(test[['year']])
        X_train = pd.concat([X_train, pd.DataFrame(temp_train)],axis=1)
        X_test = pd.concat([X_test, pd.DataFrame(temp_test)],axis=1)
        for i in range(len(cat_cols)):
            X_train = make_harmonic_features(X_train, cat_cols[i], period=harmonic_features_max[i])
            X_test = make_harmonic_features(X_test, cat_cols[i], period=harmonic_features_max[i])
        
    #X_train = pd.concat([X_train, train[new_cols]], axis=1).fillna(-999)
    #X_test = pd.concat([X_test, test[new_cols]], axis=1).fillna(-999)
    
    return X_train, X_test 

def train_groupby(train, test, window, how):
    mean_values = train[['Value', 'week_day']][-window:].groupby(['week_day']).agg(how).reset_index()
    mean_values.columns = ['week_day', 'pred']
    test = pd.merge(test, mean_values, how='left', on = ['week_day'])  
    
    return test['pred'].fillna(np.mean(train['Value']))

def calc_score(pred, fact, index_mult):
    return np.sum(abs(pred-fact)) / np.sum(fact) * 10000

def train_rf(X_train, y_train, X_valid):
    rf = RandomForestRegressor(max_features='sqrt', n_estimators=180, n_jobs=-1, random_state=4224, criterion='mae')
    rf.fit(X_train, y_train, sample_weight=np.linspace(0.5, 1, X_train.shape[0]) )
    y_hat = rf.predict(X_valid)
    return y_hat


val = train[['ForecastId','Value']].groupby('ForecastId').diff()
val['Value'] = val['Value'].fillna(0)
val['Value'] = (val['Value']==0) * 1
val.columns = ['diff']
train = pd.concat([train, val], axis=1)

a = train[['ForecastId','diff']].groupby('ForecastId').apply(pd.rolling_sum, 7, min_periods=1)
a.columns = ['ForecastId', 'to_drop']
a = a['to_drop']
train = pd.concat([train, a], axis=1)

print(train.shape)
train = train[train['to_drop']<=3].reset_index(drop=True)
train = train[['Timestamp','ForecastId','Value']]
train = time_preprocess(train)
print(train.shape)

losses = []
iids = []

sub = pd.DataFrame({})
cat_cols = ['year','week_day','month','week']
harmonic_features = ['week_day','month','week']
harmonic_features_max = [6, 12, 53]

pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']

def combine_y_hats(y_hats):
    X_stack = pd.DataFrame({})
    for i in range(0, len(y_hats)):
        X_stack['stack'+str(i)] = y_hats[i]
    return X_stack

from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=False, random_state=None)

def validate_stack(X_stack, y, model):
    y_cros_val_pred = cross_val_predict(model, X_stack, y=y, cv=5, n_jobs=-1)
    return y_cros_val_pred

def validate_lgb(X_train, y_train, X_valid, y_valid):
    
    
    d1 = lgb.Dataset(X_train, y_train, weight=np.linspace(0.5, 1, X_train.shape[0]))
    d2 = lgb.Dataset(X_valid, y_valid)
    
    params = {
        'objective':'regression',    
        'metric': 'l1', 
        'learning_rate': 0.160042,
        'random_state':42,
        #'min_data':1, 'min_data_in_bin':1
    }
    
    gbm = lgb.train(params, d1, verbose_eval=None, valid_sets=d2, 
                    num_boost_round=50000, early_stopping_rounds=100)
    
    y_hat = gbm.predict(X_valid)
    opt_boost_rounds = gbm.best_iteration
    
    return y_hat, opt_boost_rounds 



def train_lgb(X_train, y_train, X_test, opt_boost_rounds):
    
    d1 = lgb.Dataset(X_train, y_train, weight=np.linspace(0.5, 1, X_train.shape[0]))
    
    params = {
        'objective':'regression',    
        'metric': 'l1', 
        'learning_rate': 0.160042,
        'random_state':42,
        #'min_data':1, 'min_data_in_bin':1
    }
    
    gbm = lgb.train(params, d1, verbose_eval=None, num_boost_round=opt_boost_rounds)
    
    y_hat = gbm.predict(X_test)

    return y_hat

def MeanEncodingTransforming(X, y, X_test, how_to_fill):
    
    # mean encoding for lgb
    
    X_train = pd.concat([X, y], axis=1)
    mean_values = X_train.groupby(X_train.columns[0]).agg(how_to_fill).to_dict()['Value']
    X_train = X_train.drop(y.columns[0], axis=1)
    X_train = X_train.replace(mean_values)
    X_test = X_test.replace(mean_values)
    
    return X_train, X_test

def cv_lr(X_stack, y, model):
    
    # cros val lasso on X_stack 
    
    y_cros_val_pred = pd.DataFrame({})
    coefs = []
    
    for train_index, test_index in kf.split(X_stack):
        X_train, X_test = X_stack.iloc[train_index], X_stack.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model_temp = model 
        model_temp.fit(X_train,y_train)
        y_hat = model_temp.predict(X_test)
        coefs.append(model_temp.coef_)
        
        temp_df = pd.DataFrame({'id': test_index, 'Value': y_hat})
        y_cros_val_pred = pd.concat([y_cros_val_pred, temp_df], axis=0)
      
    y_cros_val_pred = y_cros_val_pred.sort_values(by='id')
    
    #divide by sum - due to overfit of model, with dividing - coefs ~ weights of models
    return list(y_cros_val_pred['Value']) / np.sum(np.mean(coefs,axis=0)) 

def make_harmonic_features(x, col, period=24):
    x['sin_'+col] = np.sin(x[col] * 2 * np.pi / period)
    x['cos_'+col] = np.cos(x[col] * 2 * np.pi / period)
    x = x.drop(col, axis=1)
    return x

index_mult = 0

for i in pd.unique(train['ForecastId'])[0:50]:
    lr_stack = Lasso(alpha=1, fit_intercept=False, max_iter=3000, tol=0.0001, positive=True, random_state=424142)
    
    # prepare train and test Dfs 
    days_in_train = 9999
    X_train = train[train['ForecastId']==i].reset_index(drop=True)[-days_in_train:].reset_index(drop=True)
    up_border = X_train['Value'].quantile(0.985)
    low_border = X_train['Value'].quantile(0.015)
    X_train = X_train[(X_train['Value']<=up_border) & (X_train['Value']>=low_border)].reset_index(drop=True)
    
    X_train['Value'] = abs(X_train['Value']) 
    X_train['Value_log'] = np.log1p(X_train['Value'])
     
    # prepare 'train_v' and 'valid_v' data frames - for validation 
    X_test = pd.DataFrame({'Timestamp':pred_dates, 'ForecastId':i})
    X_test = time_preprocess(X_test)
    #X_test = pd.merge(X_test, hol, on='Timestamp', how='left')
    #X_test = pd.merge(X_test, h, on='Timestamp', how='left')
    
    obs_in_test = 35
    X_train_v = X_train[:-obs_in_test].reset_index(drop=True)
    X_valid_v = X_train[-obs_in_test:].reset_index(drop=True)
    
    # prepare features 
        # for train
    X_train_ohe, X_test_ohe = feature_preprocessing(X_train, X_test, cat_cols, cat_type='ohe') 
    X_train_meanenc, X_test_meanenc = feature_preprocessing(X_train, X_test, cat_cols, cat_type='mean_enc')
    X_train_harm, X_test_harm = feature_preprocessing(X_train, X_test, harmonic_features, cat_type='harmonic')
  
        # for validation 
    X_train_v_ohe, X_valid_v_ohe = feature_preprocessing(X_train_v, X_valid_v, cat_cols, cat_type='ohe')
    X_train_v_meanenc, X_valid_v_meanenc = feature_preprocessing(X_train_v, X_valid_v, cat_cols, cat_type='mean_enc')
    X_train_v_harm, X_valid_v_harm = feature_preprocessing(X_train_v, X_valid_v, harmonic_features, cat_type='harmonic')
    
        # group by mean
    y_hat_grby_mean = train_groupby(X_train_v, X_valid_v, window=999999, how='mean')
    y_hat_grby3_mean = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='mean')
    
        # group by median
    y_hat_grby_median = train_groupby(X_train_v, X_valid_v, window=999999, how='median')
    y_hat_grby3_median = train_groupby(X_train_v, X_valid_v, window=obs_in_test*3, how='median')
    
        # RandomForest 
    y_rf = train_rf(X_train_v_ohe, X_train_v['Value'], X_valid_v_ohe)
    y_rf_harm = train_rf(X_train_v_harm, X_train_v['Value'], X_valid_v_harm)
    y_rf_log = np.exp(train_rf(X_train_v_ohe, X_train_v['Value_log'], X_valid_v_ohe)) - 1
    y_rf_log_harm = np.exp(train_rf(X_train_v_harm, X_train_v['Value_log'], X_valid_v_harm)) - 1
    
        # LightGBM
    y_lgb, lgb_opt = validate_lgb(X_train_v_ohe, X_train_v['Value'], X_valid_v_ohe, X_valid_v['Value'])
    y_lgb_mean, lgb_mean_opt = validate_lgb(X_train_v_meanenc, X_train_v['Value'], X_valid_v_meanenc, X_valid_v['Value'])
    y_lgb_log, lgb_opt_log = validate_lgb(X_train_v_ohe, X_train_v['Value_log'], X_valid_v_ohe, X_valid_v['Value_log'])
    y_lgb_log = np.exp(y_lgb_log) -1
    y_lgb_mean_log, lgb_mean_opt_log = validate_lgb(X_train_v_meanenc, X_train_v['Value_log'], X_valid_v_meanenc, X_valid_v['Value_log'])
    y_lgb_mean_log = np.exp(y_lgb_mean_log) -1
    
    
    X_valid_stack = combine_y_hats([y_hat_grby_mean, y_hat_grby3_mean, 
                                    y_hat_grby_median, y_hat_grby3_median, 
                                    y_rf, y_rf_harm, 
                                    y_rf_log, y_rf_log_harm, 
                                   y_lgb, y_lgb_mean, 
                                   y_lgb_log, y_lgb_mean_log
                                   ])
    y_lr_hat = cv_lr(X_valid_stack, X_valid_v['Value'], lr_stack)
    
    score_grby_mean = calc_score(y_hat_grby_mean, X_valid_v['Value'], 0)
    score_grby3_mean = calc_score(y_hat_grby3_mean, X_valid_v['Value'], 0)
    score_grby_median = calc_score(y_hat_grby_median, X_valid_v['Value'], 0)
    score_grby3_median = calc_score(y_hat_grby3_median, X_valid_v['Value'], 0)
    
    score_rf = calc_score(y_rf, X_valid_v['Value'], index_mult)
    score_rf_log = calc_score(y_rf_log, X_valid_v['Value'], index_mult)
    score_lgb = calc_score(y_lgb, X_valid_v['Value'], index_mult)
    score_lgb_mean = calc_score(y_lgb_mean, X_valid_v['Value'], index_mult)
    score_lgb_log = calc_score(y_lgb_log, X_valid_v['Value'], index_mult)
    score_lgb_mean_log = calc_score(y_lgb_mean_log, X_valid_v['Value'], index_mult)
    
    score_lr_stack = calc_score(y_lr_hat, X_valid_v['Value'], index_mult)

    y_hats     = [y_hat_grby_mean, y_hat_grby3_mean, 
                  y_hat_grby_median, y_hat_grby3_median, 
                  y_rf, 
                  y_rf_log, 
                  y_lgb, y_lgb_mean, y_lgb_log, y_lgb_mean_log,
                  y_lr_hat
                 ]
    
    all_scores = [score_grby_mean, score_grby3_mean, 
                  score_grby_median, score_grby3_median, 
                  score_rf, 
                  score_rf_log, 
                  score_lgb, score_lgb_mean, score_lgb_log, score_lgb_mean_log,
                  score_lr_stack
                 ]
    
    best_score = np.min(all_scores)
    
    models_names = ['mean_all', 'mean_3', 
                    'median_all', 'median_3', 
                    'rf', 
                    'rf_log', 
                    'score_lgb', 'score_lgb_mean', 'score_lgb_log', 'score_lgb_mean_log',
                    'lr_stack'
                   ]
    
    losses.append(best_score)
    
    temp_df = pd.DataFrame({'Timestamp':pred_dates, 'ForecastId':i})
    temp_df = time_preprocess(temp_df)
    
    if (score_grby_mean==best_score):
        y_hat = train_groupby(X_train, X_test, window=999999, how='mean')
    
    if (score_grby3_mean==best_score):
        y_hat = train_groupby(X_train, X_test, window=obs_in_test*3, how='mean')
        
    if (score_grby_median==best_score):
        y_hat = train_groupby(X_train, X_test, window=999999, how='median')
    
    if (score_grby3_median==best_score):
        y_hat = train_groupby(X_train, X_test, window=obs_in_test*3, how='median')

    if (score_rf==best_score):
        y_hat = train_rf(X_train_ohe, X_train['Value'], X_test_ohe)
        
    if (score_rf_log==best_score):
        y_hat = np.exp(train_rf(X_train_ohe, X_train['Value_log'], X_test_ohe)) -1    
    
    y_hat = 0
    X_test['Value'] = y_hat
    sub = pd.concat([sub,X_test],axis=0)
    
  
    print(i, 'Best model is', models_names[np.argmin(all_scores)])
    # print('loss:', np.min(all_scores) )
    print('val score:', np.mean(losses),'+',np.std(losses) )
    print(dict(zip(models_names, all_scores)))
    print('-------------------------------')

sub = sub[['Timestamp', 'ForecastId', 'Value']]
sub.columns = ['DATE', 'ATM_ID', 'CLIENT_OUT']
sub.to_csv('submission.csv', index=False)

In [5]:
%%time
import pandas as pd 
import numpy as np 
import lightgbm as lgb

train = pd.read_csv('train.csv')

def time_preprocess(X):
    X['Timestamp'] = pd.to_datetime(X['Timestamp'])
    X['year'] = X['Timestamp'].dt.year
    X['month'] = X['Timestamp'].dt.month 
    X['week'] = X['Timestamp'].dt.week
    X['day'] = X['Timestamp'].dt.day
    X['week_day'] = X['Timestamp'].dt.weekday
    return X
train.columns = ['Timestamp','ForecastId','Value']
train = time_preprocess(train)
train = train[train.year>=2016].reset_index(drop=True)

val = train[['ForecastId','Value']].groupby('ForecastId').diff()
val['Value'] = val['Value'].fillna(0)
val['Value'] = (val['Value']==0) * 1
val.columns = ['diff']
train = pd.concat([train, val], axis=1)

a = train[['ForecastId','diff']].groupby('ForecastId').apply(pd.rolling_sum, 7, min_periods=1)
a.columns = ['ForecastId', 'to_drop']
a = a['to_drop']
train = pd.concat([train, a], axis=1)

print(train.shape)
train = train[train['to_drop']<=3].reset_index(drop=True)
train = train[['Timestamp','ForecastId','Value']]
train = time_preprocess(train)
print(train.shape)

pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']

train_mask = train.Timestamp<=pd.to_datetime('2017-07-16')
valid_mask = train.Timestamp>pd.to_datetime('2017-07-16')

def MeanEncodingTransforming(X, y, X_test, how_to_fill):
    X_train = pd.concat([X, y], axis=1)
    mean_values = X_train.groupby(X_train.columns[0]).agg(how_to_fill).to_dict()['Value']
    X_train = X_train.drop(y.columns[0], axis=1)
    X_train = X_train.replace(mean_values)
    X_test = X_test.replace(mean_values)
    return X_train, X_test

cat_cols = ['ForecastId','year','week_day','month','week','day']
X_train = train[['ForecastId', 'year', 'month','week','day','week_day','Value']][train_mask]
X_valid = train[['ForecastId', 'year', 'month','week','day','week_day','Value']][valid_mask]
y_train = train['Value'][train_mask]
y_valid = train['Value'][valid_mask]

def feature_preprocessing(train, test, cat_cols, cat_type='ohe'):
    
    # ohe or mean encoding preprocessing for the lgb 
    
    X_train, X_test = train[cat_cols].copy(), test[cat_cols].copy()

    if (cat_type=='mean_enc'):
        for j in ['mean', 'max', 'min', 'median']:
            for i in cat_cols:
                X_train[i+'_'+j], X_test[i+'_'+j] = MeanEncodingTransforming(X_train[i], train[['Value']], X_test[i], j)
                
    if (cat_type=='ohe'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        X_train = ohe.fit_transform(train[cat_cols])
        X_test = ohe.transform(test[cat_cols])
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        
    if (cat_type=='harmonic'):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        temp_train = ohe.fit_transform(train[['year']])
        temp_test = ohe.transform(test[['year']])
        X_train = pd.concat([X_train, pd.DataFrame(temp_train)],axis=1)
        X_test = pd.concat([X_test, pd.DataFrame(temp_test)],axis=1)
        for i in range(len(cat_cols)):
            X_train = make_harmonic_features(X_train, cat_cols[i], period=harmonic_features_max[i])
            X_test = make_harmonic_features(X_test, cat_cols[i], period=harmonic_features_max[i])
        
    #X_train = pd.concat([X_train, train[new_cols]], axis=1).fillna(-999)
    #X_test = pd.concat([X_test, test[new_cols]], axis=1).fillna(-999)
    
    return X_train, X_test 

max_values = train[['Value', 'ForecastId']].groupby('ForecastId').agg('max').reset_index()
max_values.columns = ['ForecastId', 'max_value']
train1 = train.copy()
train1 = pd.merge(train1, max_values, on = 'ForecastId', how='left')
train1['Value'] = train1['Value'] / train1['max_value']
train1 = train1.drop('max_value',axis=1)
train1['year_month_weekday'] = train1['year'].astype('str') + '_' + train1['month'].astype('str') + '_' + train1['week_day'].astype('str') 
features = train1[['ForecastId', 'year_month_weekday', 'Value']].groupby(['ForecastId', 'year_month_weekday']).agg('mean').reset_index()
features = features.pivot(index='ForecastId', columns='year_month_weekday', values='Value')
features = features.fillna(0).reset_index()

X_train, X_valid = feature_preprocessing(X_train, X_valid, cat_cols, cat_type='mean_enc')
X_train = pd.merge(X_train, features, how='left', on='ForecastId')
X_valid = pd.merge(X_valid, features, how='left', on='ForecastId')

# custom metric 
def lgb_rmsle_score(preds, dtrain):
    labels = dtrain.get_label()
    preds = preds
    return 'rmsle', np.sum(abs(labels-preds))/np.sum(labels)*10000, False

dtrain = lgb.Dataset(X_train,y_train)
dval = lgb.Dataset(X_valid,y_valid)



#dtrain_all = lgb.Dataset(train[['ForecastId', 'year', 'month','week','day','week_day']],
#                         train['Value'],
#                         categorical_feature=['ForecastId', 'year', 'month','week','day','week_day'])

params = {
        'objective':'regression',    
        'metric': 'l1', 
        'learning_rate': 0.050042,
        'random_state':42,
        #'min_data':1, 'min_data_in_bin':1
    }

xgb = lgb.train(params, dtrain, valid_sets=dval, feval=lgb_rmsle_score, 
              verbose_eval=50, num_boost_round=1999999, early_stopping_rounds=150)

#xgb = lgb.train(params, dtrain_all, 
#                num_boost_round=xgb.best_iteration)

#X_test = pd.DataFrame({'Timestamp':pred_dates})
#X_test = time_preprocess(X_test)

#temp = pd.DataFrame({})
#for iid in pd.unique(train.ForecastId)[0:50]:
#    test_temp = X_test[['year', 'month','week','day','week_day','Timestamp']]
#    test_temp['ForecastId'] = iid
#    test_temp['Value'] = xgb.predict(test_temp[['ForecastId', 'year', 'month','week','day','week_day']])
#    temp = pd.concat([temp, test_temp],axis=0)
#    
#temp = temp[['Timestamp','ForecastId','Value']]
#temp.columns = ['DATE', 'ATM_ID', 'CLIENT_OUT']
#temp.to_csv('submission.csv', index=False)

C:\Users\denis\Anaconda3\envs\tf2018\lib\site-packages\pandas\core\groupby.py:707: FutureWarning: pd.rolling_sum is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=7,center=False,min_periods=1).sum()
  return func(g, *args, **kwargs)


(177900, 10)
(167834, 8)
Training until validation scores don't improve for 150 rounds.
[50]	valid_0's l1: 346088	valid_0's rmsle: 8473.79
[100]	valid_0's l1: 342455	valid_0's rmsle: 8384.84
[150]	valid_0's l1: 338878	valid_0's rmsle: 8297.26
[200]	valid_0's l1: 335281	valid_0's rmsle: 8209.18
[250]	valid_0's l1: 331729	valid_0's rmsle: 8122.22
[300]	valid_0's l1: 328186	valid_0's rmsle: 8035.47
[350]	valid_0's l1: 324793	valid_0's rmsle: 7952.39
[400]	valid_0's l1: 321348	valid_0's rmsle: 7868.04
[450]	valid_0's l1: 317877	valid_0's rmsle: 7783.06
[500]	valid_0's l1: 314546	valid_0's rmsle: 7701.49
[550]	valid_0's l1: 311312	valid_0's rmsle: 7622.31
[600]	valid_0's l1: 308040	valid_0's rmsle: 7542.2
[650]	valid_0's l1: 304818	valid_0's rmsle: 7463.32
[700]	valid_0's l1: 301549	valid_0's rmsle: 7383.28
[750]	valid_0's l1: 298303	valid_0's rmsle: 7303.81
[800]	valid_0's l1: 295056	valid_0's rmsle: 7224.31
[850]	valid_0's l1: 291945	valid_0's rmsle: 7148.13
[900]	valid_0's l1: 288777	val

[7750]	valid_0's l1: 159258	valid_0's rmsle: 3899.34


KeyboardInterrupt: 